In [12]:
from __importer import current_file_dir
from app.agents.student_agent import StudentAgent

%reload_ext autoreload
%autoreload 2

In [13]:
my_agent = StudentAgent()
my_agent.get_tasks()

[{'id': 1, 'title': 'Disease', 'description': 'Disease description?'},
 {'id': 2, 'title': 'Prognosis', 'description': 'Prognosis description?'}]

In [14]:
session_data = my_agent.start_session(1)
session_data

{'session_id': 60871,
 'history': [{'role': 'teacher',
   'content': "I've been feeling increasingly short of breath and my legs have been swelling up over the past three weeks."}],
 'score': None,
 'is_end': False}

In [15]:
from app.models import ChatMessage

In [16]:
# Define the history from the previous session start
history = []
session_id = session_data["session_id"]  # From the output of start_session
# Get the initial message from the session response
initial_message = {"role": "user", "content": "Hi, doctor. I've been having trouble breathing for the past few days. It started about four days ago and seems to be getting worse. I feel short of breath, especially when I move around or try to take a deep breath. I also have this dry cough that won't go away. There's no real chest pain, but I do feel a bit uncomfortable in my chest when I breathe in deeply. I've felt more tired than usual, and I've had some mild fevers at home. I just don't feel like myself."}
history.append(ChatMessage(**initial_message))

# Define the conversation loop
max_turns = 4
conversation_result = {"session_id": session_id, "history": history}

print(initial_message["content"])

# Run the conversation loop
for turn in range(max_turns):
    print(f"\n--- Turn {turn+1} ---")
    # Check if we've reached 3 rounds (doctor, patient, doctor, patient, doctor, patient = 3 rounds)
    if turn >= 3:
        # Generate final diagnosis
        diagnosis = my_agent.generate_diagnosis(history)
        print(f"\n--- FINAL DIAGNOSIS ---")
        print(f"Doctor: {diagnosis}")
        
        # Add diagnosis to history
        diagnosis_message = ChatMessage(role="teacher", content=diagnosis)
        history.append(diagnosis_message)
        
        # End the conversation
        break
    
    # Generate and send student reply
    student_reply = my_agent.generate_reply(history)
    print(f"Student: {student_reply}")
    
    # Add student reply to history
    student_message = ChatMessage(role="student", content=student_reply)
    history.append(student_message)
    
    # Send to teacher for evaluation
    response_data = my_agent.send_reply(session_id, history)
    
    # Check for errors
    if "error" in response_data:
        print(f"Error detected: {response_data['error']}")
        
        # Create a fake teacher response to continue the conversation
        teacher_message = ChatMessage(
            role="teacher", 
            content="I understand your concerns. Can you tell me more about when these symptoms started and if anything makes them better or worse?"
        )
        history.append(teacher_message)
        print(f"Teacher (simulated): {teacher_message.content}")
    else:
        # Update history from response
        if "history" in response_data:
            current_msg_count = len(history)
            
            response_history = []
            for msg in response_data["history"]:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    response_history.append(ChatMessage(**msg))
                elif hasattr(msg, 'role') and hasattr(msg, 'content'):
                    response_history.append(msg)
            
            # Check if we got new messages in the response
            if len(response_history) > current_msg_count:
                # Only add the latest message, which should be the teacher's response
                teacher_msg = response_history[-1]
                history.append(teacher_msg)
                print(f"Teacher: {teacher_msg.content}")
            else:
                # No new messages, create a simulated one
                teacher_message = ChatMessage(
                    role="teacher", 
                    content="Thank you for that information. Can you tell me more about your medical history?"
                )
                history.append(teacher_message)
                print(f"Teacher (simulated): {teacher_message.content}")
                
        # Check if session ended
        if response_data.get("is_end", False):
            print("\nSession ended.")
            break
    
    # Update the conversation result
    conversation_result = {"session_id": session_id, "history": history}


Hi, doctor. I've been having trouble breathing for the past few days. It started about four days ago and seems to be getting worse. I feel short of breath, especially when I move around or try to take a deep breath. I also have this dry cough that won't go away. There's no real chest pain, but I do feel a bit uncomfortable in my chest when I breathe in deeply. I've felt more tired than usual, and I've had some mild fevers at home. I just don't feel like myself.

--- Turn 1 ---
Student: I need to check your vital signs to assess your current condition.  
[Vital Check]: Tool Use
Teacher: [Vital Signs: Blood pressure 158/98 mmHg, heart rate 92 bpm, respiratory rate 20/min, temperature 36.8°C, oxygen saturation 96% on room air.]  
I've also noticed swelling in both legs and get very short of breath when I walk or lie down flat.

--- Turn 2 ---
Student: I want to take a closer look at your lungs and heart. [Chest X-ray]: Tool Use
Teacher: [Chest X-ray: Cardiomegaly and mild pulmonary conges

In [18]:
print("\nConversation Summary:")
print(f"Total messages: {len(conversation_result['history'])}")

# Print final state
print("\nFinal conversation state:")
for i, message in enumerate(conversation_result['history']):
    role = message.role.capitalize()
    content_preview = message.content[:50] + "..." if len(message.content) > 50 else message.content
    print(f"{i+1}. {role}: {content_preview}")



Conversation Summary:
Total messages: 8

Final conversation state:
1. User: Hi, doctor. I've been having trouble breathing for...
2. Student: I need to check your vital signs to assess your cu...
3. Teacher: [Vital Signs: Blood pressure 158/98 mmHg, heart ra...
4. Student: I want to take a closer look at your lungs and hea...
5. Teacher: [Chest X-ray: Cardiomegaly and mild pulmonary cong...
6. Student: I want to check your heart’s rhythm and function. ...
7. Teacher: [ECG: Sinus rhythm, evidence of left ventricular h...
8. Teacher: Thank you for sharing all of this information with...
